## Mini Project # 10 - Object Tracking

In [24]:
#Object Tracking
import cv2
import numpy as np
import uinput

device = uinput.Device([
        uinput.BTN_LEFT,
        uinput.BTN_RIGHT,
        uinput.REL_X,
        uinput.REL_Y,
        ])

# Initalize camera
cap = cv2.VideoCapture(0)

# define range of purple color in HSV
# H: 0-180, S: 0-255, V: 0-255
lower_red = np.array([0,150,100])
upper_red = np.array([15,255,255])

# Create empty points array
points = []

# Get default camera window size
ret, frame = cap.read()
Height, Width = frame.shape[:2] # 480x640
frame_count = 0

while True:

    # Capture webcame frame
    ret, frame = cap.read()
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(hsv_img, lower_red, upper_red)
    #mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    # Find contours, OpenCV 3.X users use this line instead
    #  _, contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    _, contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create empty centre array to store centroid center of mass
    center =   int(Height/2), int(Width/2)

    if len(contours) > 0:
        
        # Get the largest contour and its center 
        c = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        try:
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

        except:
            center =   int(Height/2), int(Width/2)

        # Allow only countors that have a larger than 15 pixel radius
        if radius > 25:
            
            # Draw cirlce
            cv2.circle(frame, (int(x), int(y)), int(radius),(0, 0, 255), 2)
    
    if len(points) >= 5:
        points_len = len(points)
        X_cor = sum(cor[0] for cor in points[points_len-5:points_len]) /5
        Y_cor = sum(cor[1] for cor in points[points_len-5:points_len]) /5
        device.emit(uinput.REL_X, X_cor - points[len(points)-1][0])
        device.emit(uinput.REL_Y, points[len(points)-1][1] - Y_cor)
        
    points.append(center)
    
    # Display our object tracker
    frame = cv2.flip(frame, 1)
    cv2.imshow("Object Tracker", frame)

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

# Release camera and close any open windows
cap.release()
cv2.destroyAllWindows()